<h1>Trabalho de Modelagem e Simulação</h1>
<p style='font-size: 20px'>Aluno: Artur Bernardo Mallmann</p>
Importa classe de

In [1]:
from abc import ABCMeta, abstractmethod, abstractproperty
import sys
import pandas
# class IGen:
#     __metaclass__ = ABCMeta

<h2>Classe para ler CSVs</h2>

In [2]:
from projeto import MyCSVs
arquivo = MyCSVs('entradas.csv')
help(MyCSVs)


# arquivo = pandas.read_csv('entradas.csv')
arquivo.print_html()
# print(arquivo.get_item('ir',0))

Help on class MyCSVs in module projeto:

class MyCSVs(builtins.object)
 |  MyCSVs(file_name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file_name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __next__(self)
 |  
 |  get_col(self, col)
 |  
 |  get_dict(self)
 |  
 |  get_line(self, line, campos=None)
 |  
 |  get_names(self)
 |  
 |  print_html(self)
 |  
 |  where(self, campo, valor, campos=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



__start__,fichar,consultar,pagar
3,7,23,4
5,11,15,4
8,8,18,2
9,7,17,3
11,12,22,2
12,10,25,4
14,9,21,3
17,8,22,4
18,10,16,3
21,7,20,2


<h2>Gerar Valores por monte Carlo</h2>
Caso queiramos gerar valores baseado em tempos de chegada amostrados em um CSV, basta passá-lo por por parâmetro

In [3]:
from montecarlo import MonteCarlo
classes = 8 # tipos de amostras diferentes

In [4]:
#mcs é objeto de uma das classes que gera amostras, que tbm pode ser geradores aleatórios quaisquer
mcs = dict()
for en in arquivo.get_names():
    mcs.update({en:MonteCarlo(classes,arquivo.get_col(en))})
# for en in arquivo.keys():
#     mcs.update({en:MonteCarlo(classes,arquivo[en])})

2.25
classes: 8
delta: 2.25
increment: [0, 0, 2, 2, 3, 4, 4, 6, 6, 7]
freq:[0.2, 0.2, 0.4, 0.5, 0.7, 0.7, 0.8999999999999999, 0.9999999999999999]
pms:[4.125, 6.375, 8.625, 10.875, 13.125, 15.375, 17.625, 19.875]
0.625
classes: 8
delta: 0.625
increment: [0, 6, 1, 0, 7, 4, 3, 1, 4, 0]
freq:[0.3, 0.5, 0.5, 0.6, 0.8, 0.8, 0.9, 1.0]
pms:[7.3125, 7.9375, 8.5625, 9.1875, 9.8125, 10.4375, 11.0625, 11.6875]
1.25
classes: 8
delta: 1.25
increment: [6, 0, 2, 1, 5, 7, 4, 5, 0, 4]
freq:[0.2, 0.30000000000000004, 0.4, 0.4, 0.6000000000000001, 0.8, 0.9, 1.0]
pms:[15.625, 16.875, 18.125, 19.375, 20.625, 21.875, 23.125, 24.375]
0.25
classes: 8
delta: 0.25
increment: [7, 7, 0, 4, 0, 7, 4, 7, 4, 0]
freq:[0.3, 0.3, 0.3, 0.3, 0.6, 0.6, 0.6, 1.0]
pms:[2.125, 2.375, 2.625, 2.875, 3.125, 3.375, 3.625, 3.875]


In [5]:
mc = MonteCarlo(10,[1,2,3,10])
# MonteCarlo.__str__ = lambda self:f'{self.get_rand()}'

0.9
classes: 10
delta: 0.9
increment: [0, 1, 2, 9]
freq:[0.25, 0.5, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 1.0]
pms:[1.45, 2.35, 3.25, 4.15, 5.05, 5.95, 6.8500000000000005, 7.75, 8.649999999999999, 9.549999999999999]


In [6]:
print(mcs['__start__'].get_rand())

13.125


In [7]:
print(arquivo.get_names())
# print(arquivo.keys())

['__start__', 'fichar', 'consultar', 'pagar']


<h2>Replay: Ressimular baseado em um CSV</h2>
Caso queira rever uma simulação específica basta passar o CSV por parâmetro.

In [8]:
from projeto import IGen

class Replay(IGen):
    def __init__(self,inputs):
        self.inputs = list(inputs)
    def get_rand(self):
        return self.inputs.pop(0) if len(self.inputs)!=0 else -1 #sempre retira o primeiro

In [9]:
# file_name = 'entradas.csv'
classes = 8
replay = dict()
for en in arquivo.get_names():
    col = arquivo.get_col(en)
    replay.update({en:Replay(arquivo.get_col(en))})
# for en in arquivo.keys():
#     replay.update({en:Replay(arquivo[en])})

In [10]:
print(replay['__start__'].get_rand())

3


<h2>Rand: Entradas randômicas</h2>
Gera os números de entrada de forma randômica.

In [11]:
import random
class GenRand(IGen):
    def __init__(self,mini,maxi):
        self.bounds=(mini,maxi)
    def get_rand(self):
        return random.uniform(*self.bounds)
aaa = GenRand(2,5)
print(aaa.get_rand())

3.4554655038170403


<h1>Simulador SimFlex</h1>
O simulador <b>SimFlex</b> tem o intuito de ser uma ferramenta totalmente flexível e configurável pelo usuário.
Os arquivos de configurações descrevem totalmente o funcionamento da simulação. Eles são:
entidades.csv, operacoes.csv e entidades.csv(opcional)

<b>entidades.csv:</b> Arquivo responsável por configurar a alocação de Recursos e dados como tempo de execução em cada processo lógico.

In [12]:
entidades = MyCSVs('entidades.csv')
entidades.print_html()
# entidades = pandas.read_csv('entidades.csv')
# entidades

LP,serv,acoes,tmin,tmax
__init__,1,__start__,2,5
balcao,2,fichar,7,12
balcao,2,pagar,2,4
clinica,3,consultar,15,25
__end__,0,__stop__,0,0


<b>operacoes.csv:</b> Este arquivo é responsável por descrever qual o fluxo das entidades dentro do sistema

In [13]:
operacoes = MyCSVs('operacoes.csv')
operacoes.print_html()
# operacoes = pandas.read_csv('operacoes.csv')
# operacoes

LP,value,dest,nvalue
__init__,__start__,balcao,fichar
balcao,fichar,clinica,consultar
balcao,pagar,__end__,__stop__
clinica,consultar,balcao,pagar


<b>entradas.csv</b>(opcional)<b>:</b> Arquivo com a descrição de tempos de eventos pré definidos para gerar novos tempos por monte carlo ou para reproduzir um replay.

In [14]:
entradas = MyCSVs('entradas.csv')
entradas.print_html()
# entradas = pandas.read_csv('entradas.csv')
# entradas

__start__,fichar,consultar,pagar
3,7,23,4
5,11,15,4
8,8,18,2
9,7,17,3
11,12,22,2
12,10,25,4
14,9,21,3
17,8,22,4
18,10,16,3
21,7,20,2


For each funcionando em todos:

In [15]:
for i in entidades:
    print(i)
for i in entradas:
    print (i)
for i in operacoes:
    print (i)
    
operacoes.where('LP','balcao')

('__init__', 1, '__start__', 2, 5)
('balcao', 2, 'fichar', 7, 12)
('balcao', 2, 'pagar', 2, 4)
('clinica', 3, 'consultar', 15, 25)
('__end__', 0, '__stop__', 0, 0)
(3, 7, 23, 4)
(5, 11, 15, 4)
(8, 8, 18, 2)
(9, 7, 17, 3)
(11, 12, 22, 2)
(12, 10, 25, 4)
(14, 9, 21, 3)
(17, 8, 22, 4)
(18, 10, 16, 3)
(21, 7, 20, 2)
('__init__', '__start__', 'balcao', 'fichar')
('balcao', 'fichar', 'clinica', 'consultar')
('balcao', 'pagar', '__end__', '__stop__')
('clinica', 'consultar', 'balcao', 'pagar')


[('fichar', 'clinica', 'consultar'), ('pagar', '__end__', '__stop__')]

Mensagems e servidores

In [16]:
class Msg:
    def __init__(self,time,destino,action):
        self.time=time #relógio do processo
        self.destino=destino
        self.action=action # ir,fichar, consultar, pagar, sair
    def run(self,time,action):
        self.time+=time
        self.action=action
    def __str__(self):
        return f'{self.action}({self.time})\n'
    def __repr__(self):
        return self.__str__()
    
class Spv: #spv service providers (atendentes) ou pode ser usado como fila quando sem entrada.
    def __init__(self, msg, timer):
#         msg.time+=timer.get_rand() # Calcula novo tempo para a mensagem
#         msg.action=
        pass
    def atende(self):
        pass


Processo Lógico, responsável por simular a cadeia de eventos de forma que os eventos desencadeiem na ordem correto, mesmo que assíncrono.

In [81]:
class LogicalProcess:
    def __init__(self,lpid,serv):
        self.lpid = lpid
        self.fila=[] # A princípio filas únicas
        self.serv=serv # entidades simultaneas no LP
        self.timers=dict()
        self.routes=dict()
        self.lvc = 0 #relógio local
        self.entities = []
        self.acoes = []
    def load (self):
        # sort by time
        self.fila.sort(key=lambda a:a.time)
        print(self.fila)
        if len(self.fila)>0 and len(self.entities)<self.serv:
            msg = self.fila.pop()
            t = self.timers[msg.get_action()].get_rand()
            nxt,action = self.routes[msg.action]
            msg.run(t,nxt,action)
            self.entities.append(msg) #fifo
    def set_timer(self,action,timer):
        self.timers.update({action:timer})
    def qeue(self,msg):
        if (self.lvc <= msg.time):
            self.fila.append(msg)
    def run(self):
        print("%"*50)
        self.load()
        self.entities.sort(key=lambda a:a.time)
        if len(self.entities):
            msg = self.entities[0]
            self.lvc = msg.time #seta o menor tempo local
            if(msg.destino)
            destino.qeue(msg)
    def set_route(self,action, destino,naction):
        self.routes.update({action:(destino,naction)})
    def notify(self,msg):
        pass
    def __str__(self):
        output = "="*50+"\n"
        output += f"LP: {self.lpid} (clock: {self.lvc})\nfila: {self.fila}\n"
        output += f"carga: "
        for en in self.entities:
            dest = en.action[0]
            output += f" {dest.lpid}\n"
        output +=f"( {len(self.entities)} / {self.serv} )\nrotas:\n"
        for k in self.routes.keys():
            output += f" {k}:"
            dest,na = self.routes[k]
            output += f" {dest.lpid} - {na}\n"
        return output
    def __repr__(self):
        return self.__str__()

In [82]:
tempo_global=0

class Simulation:
    def __init__ (self,entidades,operacoes,entradas=None,classes=8):
        tempo_global=0 #Menor tempo global
        
        self.lps=dict()
        # cria os processadores lógicos e já carrega a configuracao de tempo
        for entidade in entidades:
            lpid,serv,action,tmin,tmax = entidade
            lp = None
            try: #se já existir adiciona acoes à classe
                lp = self.lps[lpid]
            except:
                lp = LogicalProcess(lpid,serv)
                
            if(entradas == None):
                lp.set_timer(action,GenRand(tmin,tmax))
            elif classes==0: # Se Classes == 0 irá simular conforme os tempos do arquivo
                lp.set_timer(action,Replay(entradas.get_col(action)))
            else: # Gera tempos por monte carlo
                lp.set_timer(action,MonteCarlo(classes,entradas.get_col(action)))
                    
            self.lps.update({lpid:lp})

    #loop posterior a criação das referencias
#         self.acoes=dict()
        for op in operacoes:
            opid,value, dest, nvalue = op
            self.lps[opid].set_route(value,self.lps[dest],nvalue) # conforme value já associa nvalue e destino
#             entidades = 
#             self.acoes[k]=operacoes.where('LP',k)
    def logerr(self):
        [print(self.lps[lp],file=sys.stderr) for lp in self.lps.keys()]
    def log(self):
        [print(self.lps[lp]) for lp in self.lps.keys()]
# __init__ é o ponto de entrada
    def start(self,n):
        for i in range(n):
            self.lps['__init__'].qeue(Msg(0,'__init__','__start__'))
            
        for i in range(50): # começar limitando ciclos
            print("$"*50)
            for k in self.lps.keys():
                self.lps[k].run()
                print (f'{k}\n')
                self.log()

In [83]:
# csvs: entidades,operacoes,entradas:
entradas.get_col('__start__')

[3, 5, 8, 9, 11, 12, 14, 17, 18, 21]

In [84]:
# sim = Simulation(entidades,operacoes,entradas,10)# monte carlo
# sim = Simulation(entidades,operacoes,entradas,0) # não estocástico (só para depurações)
sim = Simulation(entidades,operacoes) #tempos uniformes
sim.log()

LP: __init__ (clock: 0)
fila: []
carga: ( 0 / 1 )
rotas:
 __start__: balcao - fichar

LP: balcao (clock: 0)
fila: []
carga: ( 0 / 2 )
rotas:
 fichar: clinica - consultar
 pagar: __end__ - __stop__

LP: clinica (clock: 0)
fila: []
carga: ( 0 / 3 )
rotas:
 consultar: balcao - pagar

LP: __end__ (clock: 0)
fila: []
carga: ( 0 / 0 )
rotas:



In [85]:
sim.start(5)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
[__start__(0)
, __start__(0)
, __start__(0)
, __start__(0)
, __start__(0)
]
__init__

LP: __init__ (clock: 3.387375397944469)
fila: [__start__(0)
, __start__(0)
, __start__(0)
, __start__(0)
]
carga:  balcao
( 1 / 1 )
rotas:
 __start__: balcao - fichar

LP: balcao (clock: 0)
fila: []
carga: ( 0 / 2 )
rotas:
 fichar: clinica - consultar
 pagar: __end__ - __stop__

LP: clinica (clock: 0)
fila: []
carga: ( 0 / 3 )
rotas:
 consultar: balcao - pagar

LP: __end__ (clock: 0)
fila: []
carga: ( 0 / 0 )
rotas:

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
[]
balcao

LP: __init__ (clock: 3.387375397944469)
fila: [__start__(0)
, __start__(0)
, __start__(0)
, __start__(0)
]
carga:  balcao
( 1 / 1 )
rotas:
 __start__: balcao - fichar

LP: balcao (clock: 0)
fila: []
carga: ( 0 / 2 )
rotas:
 fichar: clinica - consultar
 pagar: __end__ - __stop__

LP: clinica (clock: 0)
fila: []
carga: ( 0 / 3 )


In [38]:
sim.log()

LP: __init__ (clock: 0)
qeue: [__start__(0)
, __start__(0)
, __start__(0)
, __start__(0)
]
load: balcao
( {len(self.entities)} / {self.serv} )
rotas:
 __start__: balcao - fichar

LP: balcao (clock: 0)
qeue: []
load: ( {len(self.entities)} / {self.serv} )
rotas:
 fichar: clinica - consultar
 pagar: __end__ - __stop__

LP: clinica (clock: 0)
qeue: []
load: ( {len(self.entities)} / {self.serv} )
rotas:
 consultar: balcao - pagar

LP: __end__ (clock: 0)
qeue: []
load: ( {len(self.entities)} / {self.serv} )
rotas:

